In [1]:
%pip install -qU pypdf langchain_community

Note: you may need to restart the kernel to use updated packages.


In [2]:
from langchain_community.document_loaders import PyPDFLoader

file_path = "k21.pdf"
loader = PyPDFLoader(file_path)

docs = loader.load()

print(len(docs))

3


In [3]:
docs

[Document(metadata={'producer': 'PyFPDF 1.7.2 http://pyfpdf.googlecode.com/', 'creator': 'PyPDF', 'creationdate': 'D:20240812131928', 'source': 'k21.pdf', 'total_pages': 3, 'page': 0, 'page_label': '1'}, page_content="Everything About Amazon Q\nIntroduction\nOverview of Amazon Q: This section provides a brief introduction to Amazon Q, explaining what it is,\nits primary functions, and its significance in the market. It sets the stage for the rest of the document.\nPurpose and Scope of the Document: This part outlines the objectives of the document and what it\naims to cover. It defines the audience and the key topics that will be discussed.\nAmazon Q Business Overview\nBackground and History: A look into the origins of Amazon Q, including its development, launch,\nand evolution over time. This section may also cover key milestones and significant events in its\nhistory.\nMarket Position and Competitors: Analysis of Amazon Q's position within the market, including a\ncomparison with its

In [4]:
# %pip install --upgrade langsmith

import os

print("Langchain key to start tracing")
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "pr-aching-poisoning-52"
os.environ["LANGCHAIN_API_KEY"] = "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"

Langchain key to start tracing


In [5]:
from getpass import getpass
import boto3


AWS_REGION = "us-east-1"

bedrock_client = boto3.client(
    service_name="bedrock-runtime",
    region_name=AWS_REGION,
)

In [6]:
from langchain_aws import BedrockLLM

llm = BedrockLLM(client=bedrock_client,model_id="amazon.titan-text-express-v1")

In [7]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=100, chunk_overlap=15)
all_splits = text_splitter.split_documents(docs)


In [8]:
all_splits

[Document(metadata={'producer': 'PyFPDF 1.7.2 http://pyfpdf.googlecode.com/', 'creator': 'PyPDF', 'creationdate': 'D:20240812131928', 'source': 'k21.pdf', 'total_pages': 3, 'page': 0, 'page_label': '1'}, page_content='Everything About Amazon Q\nIntroduction'),
 Document(metadata={'producer': 'PyFPDF 1.7.2 http://pyfpdf.googlecode.com/', 'creator': 'PyPDF', 'creationdate': 'D:20240812131928', 'source': 'k21.pdf', 'total_pages': 3, 'page': 0, 'page_label': '1'}, page_content='Overview of Amazon Q: This section provides a brief introduction to Amazon Q, explaining what it'),
 Document(metadata={'producer': 'PyFPDF 1.7.2 http://pyfpdf.googlecode.com/', 'creator': 'PyPDF', 'creationdate': 'D:20240812131928', 'source': 'k21.pdf', 'total_pages': 3, 'page': 0, 'page_label': '1'}, page_content='what it is,'),
 Document(metadata={'producer': 'PyFPDF 1.7.2 http://pyfpdf.googlecode.com/', 'creator': 'PyPDF', 'creationdate': 'D:20240812131928', 'source': 'k21.pdf', 'total_pages': 3, 'page': 0, 'pag

In [11]:
from langchain.vectorstores import Chroma
from langchain_aws.embeddings.bedrock import BedrockEmbeddings

vectorstore = Chroma.from_documents(documents=all_splits, embedding=BedrockEmbeddings(client=bedrock_client,model_id="amazon.titan-embed-text-v2:0"))

In [12]:
retriever = vectorstore.as_retriever()

In [13]:
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 3})

retrieved_docs = retriever.invoke("what is the expiry date?")

len(retrieved_docs)

3

In [14]:
print(retrieved_docs[0].page_content)
print('---------')
print(retrieved_docs[1].page_content)
print('---------')
print(retrieved_docs[2].page_content)


history.
---------
history.
---------
Innovations in the Pipeline: A look at the upcoming features, tools, or updates planned for Amazon


In [15]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise. Don't answer anything based on your "
    "prior knowledge only answer with information from document."
    "Don't avoid even personal information from the document."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)


question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

results = rag_chain.invoke({"input": "what is the expiry date?"})

results

Error in LangChainTracer.on_llm_end callback: TracerException('No indexed run ID bf90075e-8ff4-4e56-878d-20603b260822.')


{'input': 'what is the expiry date?',
 'context': [Document(metadata={'creationdate': 'D:20240812131928', 'creator': 'PyPDF', 'page': 0, 'page_label': '1', 'producer': 'PyFPDF 1.7.2 http://pyfpdf.googlecode.com/', 'source': 'k21.pdf', 'total_pages': 3}, page_content='history.'),
  Document(metadata={'creationdate': 'D:20240812131928', 'creator': 'PyPDF', 'page': 0, 'page_label': '1', 'producer': 'PyFPDF 1.7.2 http://pyfpdf.googlecode.com/', 'source': 'k21.pdf', 'total_pages': 3}, page_content='history.'),
  Document(metadata={'creationdate': 'D:20240812131928', 'creator': 'PyPDF', 'page': 2, 'page_label': '3', 'producer': 'PyFPDF 1.7.2 http://pyfpdf.googlecode.com/', 'source': 'k21.pdf', 'total_pages': 3}, page_content='Innovations in the Pipeline: A look at the upcoming features, tools, or updates planned for Amazon')],
 'answer': '\nThe model cannot identify what the prompt is asking, and no search results are available for the question.'}